In [1]:
import numpy as np
import pandas as pd
import scipy.stats  as stats
from collections import deque
import time
from tqdm import tqdm

# reduce memory
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

# name: name of csv.gz

def naive_forecast(prediction, name):
    def get_couple_group_preds_val(pred, level1, level2):
        df = pred.groupby([level1, level2])[cols].sum()
        q = np.repeat(qs, len(df))
        df = pd.concat([df]*9, axis=0, sort=False)
        df.reset_index(inplace = True)
        y_hat = df.iloc[:,2:30].to_numpy()
        y = df.iloc[:,30:].to_numpy()
        sum_errs = np.sum((y - y_hat)**2, axis=1) # the residuals
        stdev = np.sqrt(1/(len(y)-2) * sum_errs) # the sd of residuals
        # stdev *= 0.5
        df[pred_cols] += (ratios[q]*stdev)[:, None]
        df["id"] = [f"{lev1}_{lev2}_{q:.3f}_validation" for lev1,lev2, q in
                    zip(df[level1].values,df[level2].values, q)]
        df = df[["id"]+list(cols)]
        return df.iloc[:,:29]
    
    def get_group_preds_val(pred, level):
        df = pred.groupby(level)[cols].sum()
        q = np.repeat(qs, len(df))
        df = pd.concat([df]*9, axis=0, sort=False)
        df.reset_index(inplace = True)
        y_hat = df.iloc[:,1:29].to_numpy()
        y = df.iloc[:,29:].to_numpy()
        sum_errs = np.sum((y - y_hat)**2, axis=1) # the residuals
        stdev = np.sqrt(1/(len(y)-2) * sum_errs) # the sd of residuals
        stdev *= 0.4
        df[pred_cols] += (ratios[q]*stdev)[:, None]
        if level != "id":
            df["id"] = [f"{lev}_X_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
        else:
            df["id"] = [f"{lev.replace('_validation', '')}_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
        df = df[["id"]+list(cols)]

        return df.iloc[:,:29]

    y_hat_val = prediction.iloc[:len(prediction)//2,:] # y_hat
    y_val = pd.read_csv('./data/sales_train_evaluation.csv').iloc[:,1919:] # the y label
    df_y_val = pd.concat([y_hat_val, y_val], axis=1)
    y_hat_eval = prediction.iloc[len(prediction)//2:,:]

    cols = df_y_val.iloc[:,1:].columns
    pred_cols = df_y_val.iloc[:,1:29].columns
    qs = np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995])

    dq = deque()
    dq.append(0.0)
    for q in [0.5, 0.67, 0.95, 0.99]:
        lower, upper = stats.norm.interval(q)
        dq.appendleft(lower)
        dq.append(upper)

    ratios = pd.Series(dq, index=qs).round(3)
    
    sales_val = pd.read_csv("./data/sales_train_validation.csv")
    sub_val = df_y_val.merge(sales_val[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]], on = "id")
    sub_val["_all_"] = "Total"

    sales_eval = pd.read_csv("./data/sales_train_evaluation.csv")
    sub_eval = y_hat_eval.merge(sales_eval[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]], on = "id")
    sub_eval["_all_"] = "Total"

    levels = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "_all_"]
    couples = [("state_id", "item_id"),  ("state_id", "dept_id"),("store_id","dept_id"),
                                ("state_id", "cat_id"),("store_id","cat_id")]
    df_val = []
    for level in levels :
        df_val.append(get_group_preds_val(sub_val, level))
        #df_eval.append(get_group_preds_eval(sub_eval, level, stdev))
    for level1,level2 in couples:
        df_val.append(get_couple_group_preds_val(sub_val, level1, level2))
        #df_eval.append(get_couple_group_preds_eval(sub_eval, level1, level2, stdev))
    df_val = pd.concat(df_val, axis=0, sort=False)
    df_val.reset_index(drop=True, inplace=True)
    df = pd.concat([df_val, df_val] , axis=0, sort=False)
    df.reset_index(drop=True, inplace=True)
    df.loc[df.index >= len(df.index)//2, "id"] = df.loc[df.index >= len(df.index)//2, "id"].str.replace(
                                            "_validation", "_evaluation")
    df =reduce_mem(df)
    df.to_csv(name + ".csv.gz", index = False, compression='gzip')
    
# ensembel_result = pd.read_csv('submission_accuray3.csv.gz', compression='gzip')
ensembel_result = pd.read_csv('submission_test_6.csv')
naive_forecast(ensembel_result, 'residual_v4')

-- Mem. usage decreased to 47.07 Mb (72.4% reduction),time spend:0.01 min


In [2]:
import pandas as pd

In [3]:
magic = pd.read_csv('submission_uncertainty_test_1.csv')

In [4]:
residual = pd.read_csv('residual_v4.csv.gz', compression='gzip')

In [5]:
magic.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,...,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000
mean,10.388919,9.764787,9.669005,9.732119,10.952746,12.939482,13.380063,11.664713,11.260943,10.435256,...,11.545662,13.889824,14.074576,10.867594,10.149304,10.010079,10.024647,11.120575,13.242224,13.181458
std,293.187101,274.922625,271.912069,273.642626,307.456355,363.549387,378.509086,330.310126,318.891769,294.707036,...,325.572943,393.174248,400.792566,307.544457,286.334131,281.996935,282.262683,312.779096,373.421705,373.745400
min,0.002141,0.002088,0.002323,0.002327,0.002041,0.002363,0.002593,0.002413,0.002940,0.002894,...,0.002678,0.002768,0.002777,0.002658,0.002644,0.002650,0.002652,0.002690,0.002780,0.002790
25%,0.247059,0.238041,0.237498,0.239260,0.263645,0.298752,0.306397,0.266278,0.259307,0.250433,...,0.276725,0.315930,0.316771,0.259246,0.249263,0.248870,0.249839,0.272927,0.311659,0.310000
50%,0.739159,0.704110,0.700538,0.704034,0.774758,0.897790,0.930606,0.805077,0.778701,0.743674,...,0.818144,0.956846,0.962416,0.775156,0.736204,0.731325,0.730990,0.799477,0.933111,0.930351
75%,2.157067,2.028893,2.013470,2.022250,2.247780,2.650225,2.750374,2.389069,2.293535,2.157770,...,2.375042,2.838293,2.869253,2.258101,2.113337,2.095587,2.093323,2.299227,2.730994,2.727234
max,44839.925150,42146.088463,41732.681168,42005.091928,47273.473632,55848.486370,57750.090342,50346.417977,48603.695898,45039.922724,...,49832.576091,59950.284553,60747.698092,46905.949116,43805.716151,43204.802497,43267.681146,47997.845384,57155.158523,56892.884487


In [6]:
residual.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,...,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000,771120.000000
mean,10.388973,9.764638,9.669049,9.732193,10.952753,12.939699,13.380007,11.664650,11.260606,10.435430,...,11.545461,13.889474,14.074604,10.867595,10.149068,10.010342,10.024856,11.120481,13.242182,13.181547
std,295.740397,278.170291,275.291215,276.958208,309.517058,363.869931,378.355690,331.590656,320.511725,297.207333,...,326.998054,392.604531,400.007477,309.580233,289.123816,284.990808,285.249303,314.660463,373.415458,373.728437
min,-2962.000000,-3040.000000,-3046.000000,-3018.000000,-2764.000000,-2360.000000,-2320.000000,-2876.000000,-2962.000000,-3006.000000,...,-2738.000000,-2316.000000,-2330.000000,-2926.000000,-3004.000000,-3000.000000,-2976.000000,-2730.000000,-2298.000000,-2314.000000
25%,0.344000,0.332500,0.331800,0.334000,0.367700,0.417500,0.429200,0.371300,0.363300,0.349600,...,0.384500,0.441200,0.442900,0.362300,0.349825,0.348400,0.348900,0.380100,0.434000,0.432600
50%,0.844700,0.799300,0.795400,0.798300,0.879000,1.030500,1.067000,0.923300,0.885700,0.845000,...,0.927700,1.098000,1.109000,0.885300,0.833500,0.827600,0.827000,0.903300,1.069000,1.067000
75%,2.232000,2.105000,2.084000,2.092000,2.328000,2.742500,2.854000,2.488000,2.390000,2.238000,...,2.455000,2.945000,2.986000,2.334000,2.186000,2.168000,2.166000,2.385000,2.826000,2.832000
max,44700.000000,42500.000000,42140.000000,42370.000000,46720.000000,53820.000000,55400.000000,49280.000000,47800.000000,44860.000000,...,48830.000000,57200.000000,57860.000000,46430.000000,43840.000000,43360.000000,43420.000000,47330.000000,54900.000000,54700.000000


In [9]:
residual.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
771115,WI_2_HOBBIES_0.995_evaluation,632.5,629.5,632.5,638.5,669.0,688.0,680.0,676.0,660.5,...,664.5,686.5,649.0,634.5,632.5,639.0,640.5,670.0,691.0,648.5
771116,WI_2_HOUSEHOLD_0.995_evaluation,1489.0,1472.0,1479.0,1518.0,1873.0,2064.0,1902.0,1785.0,1777.0,...,1719.0,1783.0,1616.0,1485.0,1449.0,1462.0,1509.0,1697.0,1783.0,1600.0
771117,WI_3_FOODS_0.995_evaluation,2816.0,2740.0,2684.0,2666.0,3028.0,3490.0,3502.0,3636.0,3600.0,...,3414.0,4276.0,4228.0,3182.0,2940.0,2858.0,2850.0,3122.0,3548.0,3466.0
771118,WI_3_HOBBIES_0.995_evaluation,660.0,659.0,660.5,659.5,714.5,716.5,692.0,682.0,679.0,...,720.5,725.0,691.5,667.5,665.5,667.5,664.0,719.5,723.0,689.5
771119,WI_3_HOUSEHOLD_0.995_evaluation,1400.0,1363.0,1359.0,1373.0,1566.0,1700.0,1705.0,1565.0,1518.0,...,1556.0,1654.0,1603.0,1407.0,1368.0,1366.0,1381.0,1556.0,1673.0,1610.0


In [10]:
magic.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
771115,WI_2_HOBBIES_0.995_evaluation,347.350150,343.215411,347.357233,355.994595,398.796017,425.643616,414.052807,408.841240,386.785883,...,392.294205,423.016875,370.731824,350.728692,347.556301,356.532661,359.158074,400.355860,429.528575,369.788562
771116,WI_2_HOUSEHOLD_0.995_evaluation,1290.435747,1265.958489,1276.499896,1330.487176,1828.512415,2097.679474,1870.305639,1704.970506,1694.437933,...,1613.240272,1702.239785,1469.147648,1284.232994,1234.119604,1251.812847,1317.911780,1581.788888,1702.247382,1446.062462
771117,WI_3_FOODS_0.995_evaluation,3040.280696,2933.356942,2855.910919,2829.320145,3338.353134,3985.615442,4001.853570,4190.561263,4139.219982,...,3878.265904,5089.543481,5020.900140,3552.482609,3214.778342,3099.531202,3087.041421,3470.262982,4067.626998,3953.002196
771118,WI_3_HOBBIES_0.995_evaluation,330.102225,328.181803,330.520136,329.334149,406.043054,409.102471,374.804186,360.451304,356.500412,...,414.460971,421.235077,374.024096,340.293307,337.659519,340.403830,335.235812,413.454356,418.273549,371.482744
771119,WI_3_HOUSEHOLD_0.995_evaluation,925.940682,874.904406,869.281622,888.181946,1159.868735,1348.078306,1354.025061,1157.787678,1091.811318,...,1144.966774,1283.268479,1211.256657,936.814332,882.170192,879.419636,899.470809,1145.254856,1309.172476,1221.784996
